In [2]:
import os
os.chdir('/home/sebastiaan/PhD/Repositories/immune_response_detection/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from raptcr.hashing import Cdr3Hasher
from raptcr.indexing import IvfIndex
from raptcr.constants.datasets import sample_cdr3s, example

from multiprocessing import Pool
from timeit import default_timer as timer

## Configure the search parameters

In [3]:
hasher = Cdr3Hasher(m=256).fit()

# def search_radius(seq, index:IvfIndex=index):
#     return len()

### Foreground

In [30]:

# index = IvfIndex(hasher=hasher, n_centroids=1000, n_probe=10)

# starttime = timer()
# index.add(example)
# print(f'Time elapsed BUILDING the index: {np.round(timer()-starttime, 2)} seconds')

# starttime = timer()
# foreground = {seq:index.radius_search(seq, r=0.1, exclude_self=True) for seq in example}
# print(f'Time elapsed SEARCHING the index: {np.round(timer()-starttime, 2)} seconds (NO MULTIPROCESSING)')

# pool = Pool(processes=12)

# starttime = timer()
# counts = pool.map(search_radius, example)
foreground = find_neighbors_within_radius(
        query=example,
        dataset=example
        )

seq_with_nbrs = {i:foreground[i] for i in foreground if foreground[i] > 0}

# seq_with_nbrs = {i:len(foreground[i]) for i in foreground if len(foreground[i]) > 0}

Time elapsed BUILDING the index: 3.88 seconds
Time elapsed SEARCHING the index: 25.36 seconds (WITH MULTIPROCESSING)


In [35]:
bg_counts = {i:[] for i in seq_with_nbrs}
bg_counts

{'CASSLSGAGNTIYF': [],
 'CASSLLAGPYNEQFF': [],
 'CASSTTGGYEQYF': [],
 'CASSLLNLGTDTQYF': [],
 'CASSLGGGTDTQYF': [],
 'CASSLGLAGNEQFF': [],
 'CASSLGAGGETQYF': [],
 'CASSYVGGGNQPQHF': [],
 'CASSLMDLNTGELFF': [],
 'CASSRDRNYGYTF': [],
 'CASSRTGGTNTEAFF': [],
 'CASSRAGFYGYTF': [],
 'CASSFGGAEQFF': [],
 'CASSLEGIGNQPQHF': [],
 'CASTSANTDTQYF': [],
 'CASSLTAEQFF': [],
 'CASSLSAEQFF': [],
 'CASSLIPKGLNTGELFF': [],
 'CASSRDRVSYEQYF': [],
 'CASSFGGSSYEQYF': [],
 'CASSFDSYSNQPQHF': [],
 'CASSSRRGSTDTQYF': [],
 'CASSSSATISYNEQFF': [],
 'CASSLAGTLEKLFF': [],
 'CASSTLGSTNEKLFF': [],
 'CASSFPGTEAFF': [],
 'CASSLGGGNTIYF': [],
 'CASSLRTGATGELFF': [],
 'CASSLGSGSYEQYF': [],
 'CASSLRLGDTGELFF': [],
 'CASSLAAGGPEAFF': [],
 'CASSLVAGGSNTGELFF': [],
 'CASSLLADTSTDTQYF': [],
 'CASRSGANQETQYF': [],
 'CASWTSGDEQFF': [],
 'CASSLGGISDTQYF': [],
 'CASSGGQGSNQPQHF': [],
 'CASSLAQGRNYGYTF': [],
 'CASSLGGANTQYF': [],
 'CASSAPQGAYEQYF': [],
 'CASSFSQGARTYEQYF': [],
 'CASSLGVSHSYEQYF': [],
 'CASSLRENTEAFF': [],
 'CA

### Background

In [37]:
for i in range(100):
    
    print(i)

    starttime = timer()
    background = sample_cdr3s(s=len(example))
    print(f'Time elapsed SAMPLING: {np.round(timer()-starttime, 2)} seconds')

    res = find_neighbors_within_radius(
        query=list(seq_with_nbrs.keys()),
        dataset=background
        )

    # starttime = timer()
    # index = IvfIndex(hasher=hasher, n_centroids=1000, n_probe=10)
    # index.add(background)
    # print(f'Time elapsed BUILDING the index: {np.round(timer()-starttime, 2)} seconds')

    # starttime = timer()
    # # res = [index.radius_search(seq, r=0.2, exclude_self=True) for seq in bg_counts]
    # counts = pool.map(search_radius, background)
    # res = dict(zip(list(seq_with_nbrs.keys()),counts))
    # print(f'Time elapsed SEARCHING the index: {np.round(timer()-starttime, 2)} seconds')
    
    for i in bg_counts:
        bg_counts[i].append(res[i]) 

0
Time elapsed SAMPLING: 1.01 seconds
Time elapsed BUILDING the index: 5.58 seconds
Time elapsed SEARCHING the index: 11.0 seconds


KeyError: 'CASSLLNLGTDTQYF'

### Refinement searching

In [6]:
def nneighbors(index, seq, r, exclude_self):
    return len(index.radius_search(seq, r=r, exclude_self=exclude_self))

def avg_knn_distance(hasher, query, dataset, k:int=100):

    # Index configuration
    index = IvfIndex(hasher=hasher, n_centroids=1000, n_probe=50)
    
    # Build index
    starttime = timer()
    index.add(dataset)
    print(f'Time elapsed BUILDING the index: {np.round(timer()-starttime, 2)} seconds')

    starttime = timer()
    res = index.knn_search(query, k=k)
    print(f'Time elapsed SEARCHING the index: {np.round(timer()-starttime, 2)} seconds')

    return res

def find_neighbors_within_radius(query, dataset, hasher, ncpus:int=1, k:int=150, p:int=3):
    '''
    Parameters
    ----------
    dataset
        Vectors to store on index.
    '''
    
    pool = Pool(ncpus)
    
    # Index configuration
    index = IvfIndex(hasher=hasher, n_centroids=k, n_probe=p)
    
    # Build index
    starttime = timer()
    index.add(dataset)
    print(f'Time elapsed BUILDING the index: {np.round(timer()-starttime, 2)} seconds')
    
    starttime = timer()
    counts = [nneighbors(index, seq, r=0.1, exclude_self=True) for seq in query]
    neighbors = dict(zip(query,counts))
    print(f'Time elapsed SEARCHING the index: {np.round(timer()-starttime, 2)} seconds')

    return neighbors

In [21]:
# FOREGROUND
def find_foreground_neighbors(
    repertoire,
    hasher=None,
    eliminate_singlets:bool=True
    ):
    # If none provided, use default
    if hasher is None:
        hasher = Cdr3Hasher(m=256).fit()
    fg = find_neighbors_within_radius(query=repertoire, dataset=repertoire, hasher=hasher)
    if eliminate_singlets:
        return {i:fg[i] for i in fg if fg[i] > 0}
    else:
        return fg

def neighbor_significance(
    foreground_counts:dict,
    repertoire_size:int,
    background
    ):
    # Sequences
    q = list(foreground_counts.keys())
    # Find n neighbors of query in background
    bg = find_neighbors_within_radius(hasher=hasher, query=q, dataset=background)
    # Neighbor counts
    n_f = list(bg.values()) # in foreground
    n_b = list(foreground_counts.values()) # in background
    # Hypergeometric testing
    n = repertoire_size
    M = nc + n
    pvals = {i:hypergeom.sf(i, M, n, i+j) for i,j in zip(n_f,n_g)}
    return pd.DataFrame({'sequence':q, 'fg_n':n_f, 'bg_n':n_b, 'pvalue':pvals})

In [19]:
nf = find_foreground_neighbors(example)

Time elapsed HASHING: 4.18 seconds
Time elapsed BUILDING the index: 4.67 seconds
Time elapsed SEARCHING the index: 44.29 seconds


In [22]:
control = sample_cdr3s(100000)
sign = neighbor_significance(nf, control)

Time elapsed HASHING: 2.49 seconds
Time elapsed BUILDING the index: 2.96 seconds
Time elapsed SEARCHING the index: 15.1 seconds


NameError: name 'repertoire' is not defined

In [11]:
from scipy.stats import hypergeom

def adaptive_sampling(repertoire, nc):
    '''
    repertoire
        Foreground repertoire.
    nc
        Number of sequences to sample from the background.
    '''
    
    starttime = timer()
    control = sample_cdr3s(s=nc)
    print(f'Time elapsed SAMPLING: {np.round(timer()-starttime, 2)} seconds')

    hasher = Cdr3Hasher(m=256).fit()
    
    fg = find_neighbors_within_radius(hasher=hasher, query=repertoire, dataset=repertoire)
    tn = {i:fg[i] for i in fg if fg[i] > 0}
    q = list(tn.keys())

    bg = find_neighbors_within_radius(hasher=hasher, query=q, dataset=control)

    n = len(repertoire)
    M = nc + n

    pvals = {i:hypergeom.sf(tn[i], M, n, tn[i]+bg[i]) for i in tn}

    seq = list(tn.keys())
    fg_n = list(tn.values())
    bg_n = list(bg.values())
    p = list(pvals.values())

    return pd.DataFrame({'sequence':seq, 'fg_n':fg_n, 'bg_n':bg_n, 'pvalue':p})

data = adaptive_sampling(example, 10000)


# def hypergeom_test(fg_n, bg_n, fg_total, bg_total):

# fg_total = len(example)
# bg_total = len(control)

# for seq in example[:100]:
#     fg_n = foreground[seq]
#     bg_n = background[seq]
#     if fg_n > 1:
#         pval = hypergeom.sf(fg_n, fg_total+bg_total, fg_total, fg_n+bg_n)
#         print(seq, pval

Time elapsed SAMPLING: 1.28 seconds
Time elapsed HASHING: 3.97 seconds
Time elapsed BUILDING the index: 4.43 seconds
Time elapsed SEARCHING the index: 44.82 seconds
Time elapsed HASHING: 0.25 seconds
Time elapsed BUILDING the index: 0.32 seconds
Time elapsed SEARCHING the index: 3.5 seconds


In [14]:
data[data.pvalue > .050]

,sequence,fg_n,bg_n,pvalue
2,CASSTTGGYEQYF,9,1,0.540824
4,CASSLGGGTDTQYF,33,1,0.123687
5,CASSLGLAGNEQFF,21,2,0.597899
21,CASSSRRGSTDTQYF,7,2,0.903247
47,CASSLAGSNQPQHF,45,1,0.059146
...,...,...,...,...
63884,CASSQIRYEQYF,3,1,0.782032
63887,CASSQKETQYF,24,1,0.215085
63900,CASSSLNQQYF,1,1,0.884326
63902,CASSQDRGEQYF,10,1,0.508582


In [16]:
foreground, background, p = test

data = pd.DataFrame(
    {'sequence':list(foreground.keys()), 'fg_n':list(foreground.values()), 'bg_n':list(background.values()), 'pvalue':p}
    )

data = data[data.pvalue < .05]
data.sort_values(by='pvalue')

,sequence,fg_n,bg_n,pvalue
32996,CASSQDPGVNTEAFF,2,0,0.000000
40116,CATSRGEGQETQYF,2,0,0.000000
40115,CATSRDRTGYEQYF,1,0,0.000000
40114,CASSGRLAARETQYF,1,0,0.000000
40113,CASSVEGYEQFF,1,0,0.000000
...,...,...,...,...
16976,CASSSITDTQYF,23,14,0.049426
10971,CASSLSGQAYEQYF,23,14,0.049426
16192,CASSLAGKETQYF,23,14,0.049426
27857,CASSSTGTYEQYF,23,14,0.049426


In [18]:
print('configuration 1')
r1 = find_neighbors_within_radius(
    query=example, 
    dataset=example,
    hasher=hasher,
    k=1000,
    p=10
    )

print('configuration 2')
r2 = find_neighbors_within_radius(
    query=example, 
    dataset=example,
    hasher=hasher,
    k=100,
    p=1
    )

configuration 1
Time elapsed BUILDING the index: 4.71 seconds
Time elapsed SEARCHING the index: 30.68 seconds
configuration 2
Time elapsed BUILDING the index: 1.07 seconds
Time elapsed SEARCHING the index: 24.9 seconds


In [19]:
foreground = find_neighbors_within_radius(
    hasher=hasher,
    query=example,
    dataset=example
    )

Time elapsed HASHING: 0.84 seconds
Time elapsed BUILDING the index: 4.95 seconds
Time elapsed SEARCHING the index: 31.45 seconds


In [41]:
starttime = timer()
control = sample_cdr3s(s=500000)
print(f'Time elapsed SAMPLING: {np.round(timer()-starttime, 2)} seconds')

background = find_neighbors_within_radius(
    hasher=hasher,
    query=example,
    dataset=control
    )

Time elapsed SAMPLING: 1.13 seconds
Time elapsed HASHING: 8.0 seconds
Time elapsed BUILDING the index: 16.77 seconds
Time elapsed SEARCHING the index: 66.71 seconds


In [51]:
[i for i in foreground if foreground[i] > 0]

['CASSLSGAGNTIYF',
 'CASSLLAGPYNEQFF',
 'CASSTTGGYEQYF',
 'CASSLLNLGTDTQYF',
 'CASSLGGGTDTQYF',
 'CASSLGLAGNEQFF',
 'CASSLGAGGETQYF',
 'CASSYVGGGNQPQHF',
 'CASSLMDLNTGELFF',
 'CASSRDRNYGYTF',
 'CASSRTGGTNTEAFF',
 'CASSRAGFYGYTF',
 'CASSFGGAEQFF',
 'CASSLEGIGNQPQHF',
 'CASTSANTDTQYF',
 'CASSLTAEQFF',
 'CASSLSAEQFF',
 'CASSLIPKGLNTGELFF',
 'CASSRDRVSYEQYF',
 'CASSFGGSSYEQYF',
 'CASSFDSYSNQPQHF',
 'CASSSRRGSTDTQYF',
 'CASSSSATISYNEQFF',
 'CASSLAGTLEKLFF',
 'CASSTLGSTNEKLFF',
 'CASSFPGTEAFF',
 'CASSLGGGNTIYF',
 'CASSLRTGATGELFF',
 'CASSLGSGSYEQYF',
 'CASSLRLGDTGELFF',
 'CASSLAAGGPEAFF',
 'CASSLVAGGSNTGELFF',
 'CASSLLADTSTDTQYF',
 'CASRSGANQETQYF',
 'CASWTSGDEQFF',
 'CASSLGGISDTQYF',
 'CASSGGQGSNQPQHF',
 'CASSLAQGRNYGYTF',
 'CASSLGGANTQYF',
 'CASSAPQGAYEQYF',
 'CASSFSQGARTYEQYF',
 'CASSLGVSHSYEQYF',
 'CASSLRENTEAFF',
 'CASSLEGRGETQYF',
 'CASSLAGTGVYEQYF',
 'CASSLGGGYGYTF',
 'CASSLQGSPNYGYTF',
 'CASSPTGTDTQYF',
 'CASSLRPGQTYEQYF',
 'CASSLAGSNQPQHF',
 'CASSLQGEYYEQYF',
 'CASSLLAQGEQYF',
 'CAS

In [24]:
q = [tn for tn in example if foreground[tn] > res[tn]]

157747


53061

In [42]:
nbr_counts = {i:(foreground[i], background[i])  for i in foreground}

In [49]:
nbr_counts['CASRSGANQETQYF']

(2, 0)

In [48]:
from scipy.stats import hypergeom

# def hypergeom_test(fg_n, bg_n, fg_total, bg_total):

fg_total = len(example)
bg_total = len(control)

for seq in example[:100]:
    fg_n = foreground[seq]
    bg_n = background[seq]
    if fg_n > 1:
        pval = hypergeom.sf(fg_n, fg_total+bg_total, fg_total, fg_n+bg_n)
        print(seq, pval)

# res

CASSLSGAGNTIYF 4.66335379607846e-07
CASSLLAGPYNEQFF 5.4140047173815666e-06
CASSTTGGYEQYF 0.11150730829909787
CASSLGGGTDTQYF 0.0036341176294506484
CASSLGLAGNEQFF 3.222468974411042e-06
CASSLGAGGETQYF 1.9338935607747338e-06
CASSYVGGGNQPQHF 0.01336759016161031
CASSRDRNYGYTF 0.39266872047719764
CASSRTGGTNTEAFF 0.14724198749116527
CASSRAGFYGYTF 0.2963029692859804
CASSFGGAEQFF 0.00114039773808137
CASSLEGIGNQPQHF 7.749627299947894e-05
CASSLTAEQFF 0.0007128859970093752
CASSLSAEQFF 2.4415003295622482e-05
CASSRDRVSYEQYF 0.011067055814494969
CASSFGGSSYEQYF 0.00025379417053613535
CASSSRRGSTDTQYF 0.10721036670438983
CASSLAGTLEKLFF 0.04525259014970305
CASSTLGSTNEKLFF 0.013794323911244171
CASSFPGTEAFF 0.00759474942359124
CASSLGGGNTIYF 0.0005705339232546909
CASSLGSGSYEQYF 5.731768615934041e-06
CASSLVAGGSNTGELFF 0.010686179010388364
CASSLLADTSTDTQYF 0.013794323911244171
CASRSGANQETQYF 0.0
CASSLGGISDTQYF 2.1199044000546524e-05
CASSGGQGSNQPQHF 0.49292161850116745
CASSLAQGRNYGYTF 0.5128937500931292
CASSLGG